In [4]:
from o2o import BivariateHawkesProcessSimulator, ParameterEstimator, O2OAnalyzer
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("combined_data.csv", low_memory=False, na_values=['', ' '])
cols_to_drop = ["OFFENSE_CODE_GROUP", "UCR_PART"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")
df = df.dropna(subset=["Location", "DISTRICT"])
print(f"Cleaned dataset loaded into df, shape = {df.shape}")

Cleaned dataset loaded into df, shape = (410002, 15)


In [6]:
# Steven's function to put data in correct format
def parse_dates_multi_utc(col: pd.Series) -> pd.Series:
    s = col.astype(str).str.strip()
    s = (s.str.replace(r"[…]|\.{3}", "", regex=True)
           .str.replace(r"\s+", " ", regex=True))

    # parse with utc=True every time so results are homogeneous tz-aware
    parsed = pd.to_datetime(s, format="%Y-%m-%d", errors="coerce", utc=True)

    for fmt in [
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d %I:%M %p",
        "%m/%d/%Y",
        "%m/%d/%Y %H:%M",
        "%m/%d/%Y %H:%M:%S",
    ]:
        mask = parsed.isna()
        if mask.any():
            parsed.loc[mask] = pd.to_datetime(s[mask], format=fmt, errors="coerce", utc=True)

    # final fallback (dateutil)
    mask = parsed.isna()
    if mask.any():
        parsed.loc[mask] = pd.to_datetime(s[mask], errors="coerce", utc=True)

    return parsed  # dtype should be datetime64[ns, UTC]

In [7]:
category_column = df['OFFENSE_DESCRIPTION']
unique_offenses = category_column.dropna().unique()

M = 1 
df0 = df.copy()

cutoff1 = pd.Timestamp('2025-04-01')
cutoff2 = pd.Timestamp('2025-10-01')  

offense_list = []
spillover_list = []
decay_list = []
percentages_list = []
base_list = []

#Iterating through each unique crime type (note the second district says D4, but was set to equal E13 (correct district)
for offense in unique_offenses:
    print(offense)
    df_new = df0.loc[df0["OFFENSE_DESCRIPTION"] == offense].copy()
    if df_new.empty:
        continue

    df_new['OCCURRED_ON_DATE'] = parse_dates_multi_utc(df_new['OCCURRED_ON_DATE'])
    df_new['OCCURRED_ON_DATE'] = df_new['OCCURRED_ON_DATE'].dt.tz_convert('UTC').dt.tz_localize(None)

    df_timewindow = df_new[
        (df_new['OCCURRED_ON_DATE'] >= cutoff1) &
        (df_new['OCCURRED_ON_DATE'] <= cutoff2)
    ].copy()
    if df_timewindow.empty:
        continue

    B2 = df_timewindow[df_timewindow['DISTRICT'] == 'B2'].copy()
    D4 = df_timewindow[df_timewindow['DISTRICT'] == 'E13'].copy()
    if B2.empty or D4.empty:
        continue

    t0_B2 = B2['OCCURRED_ON_DATE'].min()
    t0_D4 = D4['OCCURRED_ON_DATE'].min()
    if pd.isna(t0_B2) or pd.isna(t0_D4):
        continue

    span_days = max(
        (B2['OCCURRED_ON_DATE'].max() - B2['OCCURRED_ON_DATE'].min()).days,
        (D4['OCCURRED_ON_DATE'].max() - D4['OCCURRED_ON_DATE'].min()).days
    )
    T = max(span_days, 1)


    B2['t'] = (B2['OCCURRED_ON_DATE'] - t0_B2).dt.total_seconds() / 86400.0
    D4['t'] = (D4['OCCURRED_ON_DATE'] - t0_D4).dt.total_seconds() / 86400.0

    data_B2 = [sorted(B2['t'].dropna().tolist())]
    data_D4 = [sorted(D4['t'].dropna().tolist())]

    #O2O model
    estimator = ParameterEstimator(data_B2, data_D4, M, T)
    fit = estimator.fit_model()
    spillover, decay = estimator.spillover_and_decays_values_and_CI()
    percentages = estimator.spillover_percentage()
    base_estimation = estimator.base_and_CI()

    offense_list.append(offense)
    spillover_list.append(spillover)
    decay_list.append(decay)
    percentages_list.append(percentages)
    base_list.append(base_estimation)


FRAUD - FALSE PRETENSE / SCHEME


12:33:32 - cmdstanpy - INFO - CmdStan start processing


chain 1:   0%|          | 0/1500 [00:00<?, ?it/s, (Warmup)]

12:33:36 - cmdstanpy - INFO - CmdStan done processing.



FRAUD - IMPERSONATION


...


MANSLAUGHTER - VEHICLE - NEGLIGENCE
FIREARM/WEAPON - ACCIDENTAL INJURY / DEATH
MURDER, NON-NEGLIGENT MANSLAUGHTER


14:03:14 - cmdstanpy - INFO - CmdStan start processing


chain 1:   0%|          | 0/1500 [00:00<?, ?it/s, (Warmup)]

14:03:14 - cmdstanpy - INFO - CmdStan done processing.



PRISONER ESCAPE / ESCAPE & RECAPTURE


In [8]:
print(len(spillover_list))
print(len(offense_list))

81
81


In [9]:
print(spillover_list)

[                         Effect  Value          95% CI
0    Online to Online Spillover  0.063  [0.002, 0.200]
1  Offline to Offline Spillover  0.076  [0.005, 0.223]
2   Online to Offline Spillover  0.078  [0.004, 0.253]
3   Offline to Online Spillover  0.077  [0.003, 0.266],                          Effect  Value          95% CI
0    Online to Online Spillover  0.217  [0.010, 0.652]
1  Offline to Offline Spillover  0.281  [0.013, 0.787]
2   Online to Offline Spillover  0.229  [0.014, 0.647]
3   Offline to Online Spillover  0.296  [0.016, 0.804],                          Effect  Value          95% CI
0    Online to Online Spillover  0.123  [0.012, 0.337]
1  Offline to Offline Spillover  0.122  [0.016, 0.310]
2   Online to Offline Spillover  0.068  [0.004, 0.185]
3   Offline to Online Spillover  0.055  [0.003, 0.159],                          Effect  Value          95% CI
0    Online to Online Spillover  0.146  [0.009, 0.444]
1  Offline to Offline Spillover  0.345  [0.019, 0.899]
2   On

In [10]:
roxbury_to_JP= []
JP_to_roxbury = []
CI_RJ = []
CI_JR =[]

#getting the values and CI for the spillover 
for df in spillover_list:
    value1 = df.loc[2, 'Value'] 
    value2 = df.loc[3, 'Value'] 
    CI_value1 = df.loc[2, "95% CI"] 
    CI_value2 = df.loc[3, "95% CI"] 
    roxbury_to_JP.append(value1)
    JP_to_roxbury.append(value2)
    CI_RJ.append(CI_value1)
    CI_JR.append(CI_value2)


#get the CI in correct format

CI_cleaned1 =[]

for row in CI_RJ:
    numbers = []
    for item in row:
        try:
            num = float(item)
            numbers.append(num)
        except:
            pass

    CI_cleaned1.append(numbers)

CI_cleaned2 =[]

for row in CI_JR:
    numbers = []
    for item in row:
        try:
            num = float(item)
            numbers.append(num)
        except:
            pass

    CI_cleaned2.append(numbers)

    

In [12]:
# Making dataframe of values (unsorted/sorted)

roxbury_to_JP = [float(s) for s in roxbury_to_JP]
JP_to_roxbury = [float(s) for s in JP_to_roxbury]

data = {'Offense': offense_list, 'Spillover Roxbury to JP': roxbury_to_JP, 'Spillover JP to Roxbury':JP_to_roxbury}

data2 = {'Offense': offense_list, 'Spillover Roxbury to JP': roxbury_to_JP}
data3 = {'Offense': offense_list,'Spillover JP to Roxbury':JP_to_roxbury}


spilloverdf = pd.DataFrame(data)
spilloverdf2 = pd.DataFrame(data2)
spilloverdf3 = pd.DataFrame(data3)


spilloverdf['CI 95% Roxbury to JP'] = CI_cleaned1
spilloverdf['CI 95% JP to Roxbury'] = CI_cleaned2
spilloverdf2['CI 95% Roxbury to JP'] = CI_cleaned1
spilloverdf3['CI 95% JP to Roxbury'] = CI_cleaned2


print(spilloverdf)
sorted_spilloverdf2 = spilloverdf2.sort_values(by='Spillover Roxbury to JP', ascending=False)
sorted_spilloverdf3 = spilloverdf3.sort_values(by='Spillover JP to Roxbury', ascending=False)

spilloverdf.to_csv('/O2O/O2O/crimespilloverJP_notsorted.csv')
sorted_spilloverdf2.to_csv('/O2O/O2O/crimespilloverJP_sortedroxbury.csv')
sorted_spilloverdf3.to_csv('/O2O/O2O/crimespilloverJP_sortedJP.csv')





                                              Offense  \
0                     FRAUD - FALSE PRETENSE / SCHEME   
1                               FRAUD - IMPERSONATION   
2                       SICK/INJURED/MEDICAL - PERSON   
3                            FORGERY / COUNTERFEITING   
4                         LARCENY THEFT FROM BUILDING   
..                                                ...   
76                                              ARSON   
77                                        BOMB THREAT   
78  RECOVERED - MV RECOVERED IN BOSTON (STOLEN IN ...   
79                              FIREARM/WEAPON - LOST   
80                 MURDER, NON-NEGLIGENT MANSLAUGHTER   

    Spillover Roxbury to JP  Spillover JP to Roxbury CI 95% Roxbury to JP  \
0                     0.078                    0.077       [0.004, 0.253]   
1                     0.229                    0.296       [0.014, 0.647]   
2                     0.068                    0.055       [0.004, 0.185]   
3      

In [33]:
import os
current_directory = os.getcwd()
print(f"Current Working Directory: {current_directory}")

Current Working Directory: /Users/meganhogan/O2O
